# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [16]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [13]:
sinasc.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [8]:
# 2) seu código aqui
columns_with_missing = sinasc.columns[sinasc.isnull().any()].tolist()
columns_with_missing

for coluna in columns_with_missing:
    missing_count = sinasc[coluna].isnull().sum()
    if missing_count > 0:
        print(f"A coluna '{coluna}' possui {missing_count} valores missing.")
    else:
        print(f"A coluna '{coluna}' não possui valores missing.")
    

A coluna 'CODESTAB' possui 115 valores missing.
A coluna 'ESTCIVMAE' possui 317 valores missing.
A coluna 'ESCMAE' possui 312 valores missing.
A coluna 'CODOCUPMAE' possui 2907 valores missing.
A coluna 'QTDFILVIVO' possui 1573 valores missing.
A coluna 'QTDFILMORT' possui 2098 valores missing.
A coluna 'GESTACAO' possui 1232 valores missing.
A coluna 'GRAVIDEZ' possui 79 valores missing.
A coluna 'PARTO' possui 49 valores missing.
A coluna 'HORANASC' possui 21 valores missing.
A coluna 'SEXO' possui 4 valores missing.
A coluna 'APGAR1' possui 96 valores missing.
A coluna 'APGAR5' possui 103 valores missing.
A coluna 'RACACOR' possui 647 valores missing.
A coluna 'IDANOMAL' possui 591 valores missing.
A coluna 'CODANOMAL' possui 26814 valores missing.
A coluna 'DTRECORIGA' possui 27028 valores missing.
A coluna 'NATURALMAE' possui 298 valores missing.
A coluna 'CODMUNNATU' possui 298 valores missing.
A coluna 'CODUFNATU' possui 298 valores missing.
A coluna 'ESCMAE2010' possui 249 valo

In [10]:
# 3) seu código aqui
colunas_de_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
df_selecionado = sinasc[colunas_de_interesse]
df_selecionado

for coluna in df_selecionado:
    missing_count = sinasc[coluna].isnull().sum()
    if missing_count > 0:
        print(f"A coluna '{coluna}' possui {missing_count} valores missing.")
    else:
        print(f"A coluna '{coluna}' não possui valores missing.")

A coluna 'LOCNASC' não possui valores missing.
A coluna 'IDADEMAE' não possui valores missing.
A coluna 'ESTCIVMAE' possui 317 valores missing.
A coluna 'ESCMAE' possui 312 valores missing.
A coluna 'QTDFILVIVO' possui 1573 valores missing.
A coluna 'GESTACAO' possui 1232 valores missing.
A coluna 'GRAVIDEZ' possui 79 valores missing.
A coluna 'CONSULTAS' não possui valores missing.
A coluna 'APGAR5' possui 103 valores missing.


In [12]:
# 4) seu código aqui 
# Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.sinasc['APGAR5'].isna():
df_selecionado['APGAR5'].dropna()
 # A coluna 'APGAR5' possui 103 valores missing.
 # Linhas: 26925



0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

In [ ]:
# 5) seu código aqui
colunas_substituir = ['ESTCIVMAE', 'CONSULTAS']
df_selecionado[colunas_substituir] = df_selecionado[colunas_substituir].replace(9, np.nan)


In [22]:
# 6) Seu código aqui Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
df_selecionado['QTDFILVIVO'].fillna(0)

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [48]:
# 7) seu código aqui
variaveis = ['CONSULTAS', 'QTDFILVIVO'] 
for coluna in variaveis:
    df_selecionado[coluna].fillna(df_selecionado[coluna].median())
    # A mediana reflete melhor o valor central das variáveis quantiativas.

In [50]:
# 8) seu código aqui
"""O Apgar possui uma classificação indicando se o bebê passou por asfixia:
Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'."""

df_selecionado.loc[(df_selecionado['APGAR5'] >= 8) & (df_selecionado['APGAR5'] <= 10), 'CATEGORIA_ASFIXIA'] = 'Normal'
df_selecionado.loc[(df_selecionado['APGAR5'] >= 6) & (df_selecionado['APGAR5'] <= 7), 'CATEGORIA_ASFIXIA'] = 'Asfixia Leve'
df_selecionado.loc[(df_selecionado['APGAR5'] >= 4) & (df_selecionado['APGAR5'] <= 5), 'CATEGORIA_ASFIXIA'] = 'Asfixia Moderada'
df_selecionado.loc[(df_selecionado['APGAR5'] >= 0) & (df_selecionado['APGAR5'] <= 3), 'CATEGORIA_ASFIXIA'] = 'Asfixia Severa'

df_selecionado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,apgar5_cat,CATEGORIA_ASFIXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,NaN,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0,NaN,Normal
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0,NaN,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0,NaN,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4.0,9.0,NaN,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0,NaN,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,NaN,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4.0,9.0,NaN,Normal


In [60]:
# 9) seu código aqui
sinasc.columns = sinasc.columns.str.lower().str.replace(' ', '_')
sinasc.columns

Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea'],
      dtype='object')